<h1>STORED FUNCTION</h1>

<h3 id="Create-sf_count_retailers">Create sf_count_retailers</h3>
<p><span style="font-size: 12pt;">This will count how many retailers this dealer is.</span></p>

In [ ]:
CREATE FUNCTION sf_count_retailers(
	cellno varchar(11)
) 
RETURNS int
DETERMINISTIC
BEGIN
	declare ctr int; 
	set ctr= (
		select 	count(*) 
		from 	e_subscribers 
		where 	upline_id in (select a.id from e_subscribers a where a.cellno=cellno) 
			and inactive=0 
			and blocked=0	
	);
	return ctr;
END


<p><span style="font-size: 12pt;">Let's put it in action...</span></p>

In [ ]:
select sf_count_retailers(cellno) as retailers, a.* from e_subscribers a

<h3>Create sf_get_current_balance</h3>
<p>This will get the current balance of the subscriber.</p>

In [ ]:
CREATE FUNCTION sf_get_current_balance(
	cellno varchar(11)
) 
RETURNS int
DETERMINISTIC
BEGIN
	declare balance  		decimal(15,2); 
	declare subscribers_id	char(36);
	set balance=0;
	set subscribers_id = (select a.id from e_subscribers a where a.cellno=cellno);
	set balance= (
		select		a.amount_beginning + a.amount_actual
		from 		e_ledger a
		where       a.e_subscribers_id=subscribers_id
		order by    created_at desc
		limit 		1
	);
	return ifnull(balance,0);
END

<p><span style="font-size: 12pt;">Let's put it in action...</span></p>

In [ ]:
select sf_get_current_balance(cellno) as retailers, a.* from e_subscribers a

<h3 id="Create-a-split">Create a split stored function</h3>
<p><span style="font-size: 12pt;">This stored function will be used in parsing the incoming message to process a valid command</span></p>

In [ ]:
CREATE FUNCTION split(x VARCHAR(65000), delim VARCHAR(12), pos INTEGER) 
RETURNS VARCHAR(65000)
DETERMINISTIC
BEGIN
  DECLARE output VARCHAR(65000);
  SET output = REPLACE(SUBSTRING(SUBSTRING_INDEX(x, delim, pos)
                 , LENGTH(SUBSTRING_INDEX(x, delim, pos - 1)) + 1)
                 , delim
                 , '');
  IF output = '' THEN SET output = null; END IF;
  RETURN output;
END

<p><span style="font-size: 12pt;">Let's put it in action...</span></p>

In [ ]:
select split('the quick brown fox jumps', ' ',1) 